In [8]:
import pandas as pd
import numpy as np

In [37]:
def read_rawimage(file,slices=np.arange(-45,45,8)):
    df = pd.read_csv(file,header=None)
    image_cols = df.iloc[:,:-4].astype(str)
    last_col = df.iloc[:,-2]/100*(180/np.pi)
    new_last_col = last_col.apply(lambda x: np.abs(x-slices).argmin())
    image = image_cols.apply(lambda col: np.array(eval(','.join(col.to_list()))).astype(float),axis=1)
    new_df = pd.DataFrame({'Image':image,'Slice':new_last_col})
    return new_df

In [38]:
fullclip = read_rawimage('data-RawImage-rpi1.csv')
emptyclip = read_rawimage('data-RawImage-rpi1-empty-0619.csv')
pipe = read_rawimage('data-RawImage-rpi1-pipe-0619.csv')

In [39]:
fullclip['Slice'].head()

0    3
1    3
2    8
3    8
4    3
Name: Slice, dtype: int64

In [50]:
def generate_augmented_images(df,n_images=1000,slices=np.arange(-45,45,8),
                              xshift=10,yshift=10,
                              xcrop=[20,80],ycrop=[0,50],zcrop=[1,11]):
    xdim,ydim = df['Images'][0].shape
    df_slices = df[df['Slice'] == slc]
    if len(df_slices) == 0:
        full_stack = np.zeros((n_images,xdim,ydim))
    else:
        df_slices = [df_slices.sample(n_images,replace=True) for slc in slices]
        full_stack = list(np.stack([np.stack(s['Image'].to_list(),axis=0) for s in df_slices],axis=3))
    
        x = np.random.randint(xshift,size=n_images)
        y = np.random.randint(yshift,size=n_images)
        for i in range(n_images):
            orig = full_stack[i]
            aug = np.zeros_like(orig)
            aug[x[i]:,y[i]:,:] = orig[:xdim-x[i],:ydim-y[i],:]

            full_stack[i] = aug[xcrop[0]:xcrop[1],ycrop[0]:ycrop[1],zcrop[0]:zcrop[1]]
    
    return np.stack(full_stack)

SyntaxError: invalid syntax (<ipython-input-50-e8e23e5a0978>, line 9)

In [72]:
def new_gen(df,n_images=1000,n_slices=12,xshift=10,yshift=10,xcrop=[20,80],ycrop=[0,50],zcrop=[1,11]):
    xdim,ydim = df['Image'][0].shape
    full_stack = []
    for slc in range(n_slices):
        df_slc = df[df['Slice']==slc]
        if len(df_slc) == 0:
            sample_slc = np.zeros((n_images,xdim,ydim))
        else:
            sample_slc = np.stack(df_slc.sample(n_images,replace=True)['Image'].to_list())
        full_stack.append(sample_slc)
        
    full_stack = np.stack(full_stack,axis=3)
    for i in range(n_images):
        orig = full_stack[i,:,:,:]
        aug = np.zeros_like(orig)
        aug[]
    
    
    return full_stack
            

In [ ]:
full_stack.shape

(5, 101, 91, 12)

In [6]:
import h5py

In [7]:
hf = h5py.File('data.h5', 'w')
hf.create_dataset('clip', data=full_stack,compression="gzip", compression_opts=9)
hf.close()